# Neural Networks - intro
## Part 1 - XOR
1. Using the XOR dataset below, train (400 epochs) a neural network (NN) using 2, 3, 4, and 5 hidden layers (where each layer has only 2 neurons). For each n layers, store the resulting accuracy along with n. Plot the results to find what the optimal number of layers is.
2. Repeat the above with 3 neurons in each Hidden layers. How do these results compare to the 2 neuron layers?
3. Repeat the above with 4 neurons in each Hidden layers. How do these results compare to the 2 and 3 neuron layers?
3. Using the most optimal configuraion (n-layers, k-neurons per layer), compare how `tanh`, `sigmoid`,`softplus` and `relu` effect the loss after 400 epochs. Try other Activation functions as well (https://keras.io/activations/)
4. Again with the most optimal setup, try other optimizers (instead of `SGD`) and report on the loss score. (https://keras.io/optimizers/)

## Part 2 - BYOD (Bring your own Dataset)
Using your own dataset, experiment and find the best Neural Network configuration. 
You may use any resource to improve results, just reference it.

While you may use any dataset, I'd prefer you didn't use the diabetes dataset used in the lesson.

https://stackoverflow.com/questions/34673164/how-to-train-and-tune-an-artificial-multilayer-perceptron-neural-network-using-k

https://keras.io/

In [1]:
!pip3 install tensorflow keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD  #Stochastic Gradient Descent


In [ ]:
import numpy as np
np.random.seed(7)
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
n = 40
xx = np.random.random((n,1))
yy = np.random.random((n,1))

In [ ]:
X = np.array([np.array([xx,-xx,-xx,xx]),np.array([yy,-yy,yy,-yy])]).reshape(2,4*n).T
y = np.array([np.ones([2*n]),np.zeros([2*n])]).reshape(4*n)

In [ ]:
plt.scatter(*zip(*X), c=y)

In [ ]:
model = Sequential()

model.add(Dense(2, input_dim=2, activation='tanh'))  #sigmoid, relu
# model.add(Dense(2, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
# model.add(Dense(1,input_dim=2, activation='sigmoid'))


sgd = SGD(lr=0.1)
model.compile(loss='binary_crossentropy', optimizer='sgd')

model.fit(X, y, batch_size=2, epochs=400) #160/4 = 40 per epoch
print(model.predict_proba(X).reshape(4*n))

# evaluate the model
scores = model.evaluate(X, y)

In [ ]:
print(model.predict_proba(X).reshape(4*n))


In [ ]:
scores = model.evaluate(X, y)
scores, model.metrics_names

In [ ]:
plt.scatter(*zip(*X), c=model.predict_classes(X))

In [ ]:
plt.scatter(*zip(*X), c=model.predict(X))

## Using Diabetes data 

http://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data

1. Number of times pregnant 
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test 
3. Diastolic blood pressure (mm Hg) 
4. Triceps skin fold thickness (mm) 
5. 2-Hour serum insulin (mu U/ml) 
6. Body mass index (weight in kg/(height in m)^2) 
7. Diabetes pedigree function 
8. Age (years) 
9. Class variable (0 or 1) 

In [ ]:
from keras.models import Sequential
import pandas as pd
from keras.utils import to_categorical
coffee = pd.read_csv('~/Desktop/df_arabica_clean.csv', delimiter=",")
coffee.head

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [ ]:
# create model
model = Sequential()
model.add(Dense(16, input_dim=8, activation='tanh'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X, Y, epochs=1000, batch_size=10)
# evaluate the model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
def byod2(layers, neurons, act, opt):
    model = Sequential()
    model.add(Dense(neurons, input_dim= 13, activation=act))
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation=act))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    record = model.fit(X, y, epochs=100, batch_size=10, validation_split=0.4, verbose=0)
    loss, accuracy = model.evaluate(X, y, verbose=0)
    return loss, accuracy, record.history

In [ ]:
configs = [
    {'layers': 2, 'neurons': 3, 'act': 'tanh', 'opt': 'Lion'},
    {'layers': 5, 'neurons': 3, 'act': 'tanh', 'opt': 'Lion'},
    {'layers': 1, 'neurons': 4, 'act': 'tanh', 'opt': 'Lion'},
    {'layers': 2, 'neurons': 3, 'act': 'selu', 'opt': 'Lion'},
    {'layers': 5, 'neurons': 3, 'act': 'selu', 'opt': 'Lion'},
    {'layers': 1, 'neurons': 4, 'act': 'selu', 'opt': 'Lion'},
    {'layers': 2, 'neurons': 3, 'act': 'tanh', 'opt': 'Adam'},
    {'layers': 5, 'neurons': 3, 'act': 'tanh', 'opt': 'Adam'},
    {'layers': 1, 'neurons': 4, 'act': 'tanh', 'opt': 'Adam'},
    {'layers': 2, 'neurons': 3, 'act': 'selu', 'opt': 'Adam'},
    {'layers': 5, 'neurons': 2, 'act': 'selu', 'opt': 'Adam'},
    {'layers': 1, 'neurons': 4, 'act': 'selu', 'opt': 'Adam'}
]

In [ ]:
results2 = []

for config in configs:
    layers = config['layers']
    neurons = config['neurons']
    act = config['act']
    opt = config['opt']
    loss, accuracy, record = byod2(layers, neurons, act,  opt)
    results2.append({'layers': layers, 'neurons': neurons, 'activation': act, 
                    'optimizater':opt, 'loss':loss, 'accuracy':accuracy, 'record': record}) 